In [55]:
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [56]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [57]:
%pip install langchain_openai langgraph langchain tavily-python --quiet

Note: you may need to restart the kernel to use updated packages.


In [58]:
%pip install -U langsmith --quiet

Note: you may need to restart the kernel to use updated packages.


# Multiply tool calling

In [59]:
import json
from langchain_core.messages import ToolMessage, BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI
from typing import List

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together."""
    return a * b

# def model, bind tools
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)
model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])

# create graph
graph = MessageGraph()

# model invoker
def invoke_model(state: List[BaseMessage]):
    return model_with_tools.invoke(state)

# entry point node
graph.add_node("oracle", invoke_model)

# 
def invoke_tool(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No adder input found")

    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
    )

    return ToolMessage(
        tool_call_id = multiply_call.get("id"),
        content=res
    )

graph.add_node("multiply", invoke_tool)
graph.add_edge("multiply", END)
graph.set_entry_point("oracle")

In [60]:
def router(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"

graph.add_conditional_edges("oracle", router, {
    "multiply": "multiply",
    "end": END,
})

In [61]:


runnable = graph.compile()

messages = runnable.invoke(HumanMessage("What is 123 * 456?"))

print(messages[-1].content)

56088


# Cycles

In [62]:
TAVILY_API_KEY = "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"
LANGCHAIN_API_KEY = "ls__02b4670bcc97473595c092908b1376df"
LANGCHAIN_TRACING_V2 = "true"

In [63]:
import os
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2

In [64]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolExecutor
from langchain_openai import ChatOpenAI
from langchain.tools.render import format_tools_openai_function

tools = [TavilySearchResults(max_results=1)]
tool_executor = ToolExecutor(tools)
model = ChatOpenAI(temperature=0, streaming=True)

functions = [format_tools_openai_function(t) for t in tools]
model = model.bind(tools=functions)

ImportError: cannot import name 'format_tools_openai_function' from 'langchain.tools.render' (/Users/brendanmclaughlin/anaconda3/envs/cs224n/lib/python3.11/site-packages/langchain/tools/render.py)

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# function that determines whether or not to continue
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # if no function to call we're finished
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # otherwse continue
    else:
        return "continue"

# the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # return a list bc it will get added to the existing list
    return {"messages": [response]}

# define function to execute tools
def call_tool(state):
    messages = state['messages']
    # based on the continue condition we know that the last message has a function call
    last_message = messages[-1]
    # construct tool invocation from function call
    action = ToolsInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    )
    # call tools executor and get back response
    response = tool_executor.invoke([action])
    # create fuction message
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # return a list bc it will get added to the existing list
    return {"messages": [function_message]}

### Define the Graph

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    # define start node which means there are edges taken after the `agent` node is called
    "agent", 
    # next, pass in the function that will determine which node is next
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# we now add a normal edge from the tool to agent 
# this means that after `tools` is called, we will call `agent` again
workflow.add_edge("action", "agent")

app = workflow.compile()

### Use it!

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in SF")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in SF'),
  AIMessage(content='Currently, the weather in San Francisco is partly cloudy with a temperature of 62°F (17°C).', response_metadata={'finish_reason': 'stop'}, id='run-fdf7a2e9-cd7b-4868-8f43-0af08909e13d-0')]}